In [1]:
import re

In [2]:
keyword_pair_map = {
    0: ["a","á"],
    1: ["ais","áis"],
    2: ["aisti","aistí"],
    3: ["ait","áit"],
    4: ["ar","ár"],
    5: ["arsa","ársa"],
    6: ["ban","bán"],
    7: ["cead","céad"],
    8: ["chas","chás"],
    9: ["chuig","chúig"],
    10: ["dar","dár"],
    11: ["do","dó"],
    12: ["gaire","gáire"],
    13: ["i","í"],
    14: ["inar","inár"],
    15: ["leacht","léacht"],
    16: ["leas","léas"],
    17: ["mo","mó"],
    18: ["na","ná"],
    19: ["os","ós"],
    20: ["re","ré"],
    21: ["scor","scór"],
    22: ["te","té"],
    23: ["teann","téann"],
    24: ["thoir","thóir"],
}

In [3]:
# load text
def readfile(filename):
    file = open(filename, 'rt')
    original_text = file.read()
    file.close()
    
    return original_text

In [4]:
def writefile(filename, content):
    file = open(filename, 'w')
    file.write(str(content))
    file.close()

In [ ]:
def preprocess_data(text):
    # remove punctuations !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    punctuation_list = ["!","\"","#","$","%","&","'","(",")","*","+",",","-",".","/",":",";","<","=",">","?","@","[","\\","]","^","_","`","{","|","}","~","–","€","•","«","»","’","“","”","£"]
    for punctuation in punctuation_list:
        text = text.replace(punctuation, "")

    # remove emojis
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    text = RE_EMOJI.sub(r'', text)

    # substitute multiple spaces with a single space
    text = re.sub(' +', ' ', text)

    # normalize text to lower case
    text = text.lower()
    
    return text

In [23]:
text = readfile('train.txt')
text = preprocess_data(text)

In [6]:
# tokenize text to a list of sentences
sentences = text.split("\n")

sentence_map = {k: [] for k in keyword_pair_map}
for sentence in sentences:
    for i, (key, keyword_pair) in enumerate(keyword_pair_map.items()):
        if(keyword_pair[0] in sentence or keyword_pair[1] in sentence):
            sentence_map[key].append(sentence);

In [7]:
prob_map = {}
for key in sentence_map:
    sentence_list = sentence_map[key]
    keyword_pair = keyword_pair_map[key]
    count_1 = 0
    count_2 = 0
    prob_1 = 0;
    prob_2 = 0
    
    for sentence in sentence_list:
        if keyword_pair[0] in sentence:
            count_1 += 1;
        if keyword_pair[1] in sentence:
            count_2 += 1

    num_sentences = len(sentence_list)
    prob_1 = count_1 / num_sentences
    prob_2 = count_2 / num_sentences
    prob_map[keyword_pair[0]] = prob_1
    prob_map[keyword_pair[1]] = prob_2

In [20]:
def generate_submission_choice():
    test_text = readfile('test.txt')

    test_sentences = test_text.split("\n");
    test_word_choices = []
    answer_list = ["Id,Expected"]
    running_number = 1
    for sentence in test_sentences:
        start_index = sentence.find("{")
        end_index = sentence.find("}")

        word_choice = sentence[start_index+1:end_index]
        if(word_choice):
            word_choice_pair = word_choice.split("|")
            answer_list.append("{},{}".format(running_number, (1- (prob_map[word_choice_pair[1]]))))
        running_number += 1
    answer = "\n".join(answer_list)
    return answer

In [22]:
submission = generate_submission_choice()
writefile('submission7.csv', submission)